In [ ]:
%%bash
pip install pandas-datareader
pip install get-all-tickers
pip install yfinance

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import History
from collections import Iterable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

import pandas as pd
import time
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame
from get_all_tickers import get_tickers as gt

#getting the labels
news_data = pd.read_csv('Combined_News_DJIA.csv', index_col=0)
labels = news_data[['Label']]

# USER INPUT REQUIRED FOR STOCK SELECTION, DATE & DECIDING TOP MOVER THRESHOLD PERCENTAGE

In [ ]:
stocks = input("Which stocks do you want to check: ")
#ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL   <-------using this as first input for test purposes(any ticker stoc symbols could be used)
date_check = input("Please enter date between 2008/08/08 and 2016/07/01 in yyyy-mm-dd format to use Market Sentiment:")
#2016-06-30        <----------- use this as second input
defined_topmover_threshold = input("Please enter % threshold to define top movers stocks:")  ## Set this to define top mover threshold percentage

Which stocks do you want to check: ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL
Please enter date between 2008/08/08 and 2016/07/01 in yyyy-mm-dd format to use Market Sentiment:2016-06-30
Please enter % threshold to define top movers stocks:10


In [ ]:
def Convert(string): 
    li = list(string.split(",")) 
    return li
stocks=Convert(stocks) 

In [ ]:
from pandas_datareader import data as pdr
from datetime import date
import yfinance as yf
yf.pdr_override()
import pandas as pd

# Tickers list
# We can add and delete any ticker from the list to get desired ticker live data
ticker_list=stocks
# We can get data by our choice by giving days bracket
start_date= '2008-08-08'
end_date=date_check
files=[]
# Create a data folder in your current dir.
def SaveData(df, filename):
  df.to_csv('./'+filename+'.csv')
def getData(ticker):
  print (ticker)
  data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
  dataname = ticker
  files.append(dataname)
  SaveData(data, dataname)

#This loop will iterate over ticker list, will pass one ticker to get data, and save that data as file.
for tik in ticker_list:
  getData(tik)

ZION
[*********************100%***********************]  1 of 1 completed
KGC
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
SOHU
[*********************100%***********************]  1 of 1 completed
BLIN
[*********************100%***********************]  1 of 1 completed
WMT
[*********************100%***********************]  1 of 1 completed
AMGN
[*********************100%***********************]  1 of 1 completed
NKTR
[*********************100%***********************]  1 of 1 completed
T
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed


In [ ]:
class fun_t(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
def dataset(xtra,x,y,time_steps):
  import numpy as np
  xs, ys = [], []
  xtra=xtra
  xnum=xtra.to_numpy()
  for i in range(len(x) - time_steps):
    v = x.iloc[i : (i+time_steps)].to_numpy()
    comb=np.concatenate((v, xnum[i+time_steps]), axis=None)
    xs.append(comb)
    ys.append(y.iloc[i+time_steps]) #So that next y is considered
  return np.array(xs), np.array(ys)

# LSTM PREDICTIONS WITHOUT SENTIMENT SCORE INFORMATION

In [ ]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL

for tik in ticker_list:
  tik_data=pd.read_csv(tik+'.csv',index_col=0)
  temp_data=tik_data.drop(['Close'],axis=1)
  
  with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)
  #Scaling the variables(x) and variable(y)
  sca = temp_data.iloc[:,:]
  #Standardization of data is required as the dataset consists of variables of different scales
  sc = StandardScaler()
  ysc = StandardScaler()
  scaled_wo_sentiment = sc.fit_transform(sca)
  scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])
  
  #Y scaling for later inverse transforming predictions
  yscalesenti = with_sentiment_data
  y_with_sentiscal = ysc.fit_transform(yscalesenti.iloc[:,4].to_numpy().reshape(-1,1))
  with_sentiment_data = sc.fit_transform(with_sentiment_data)

  with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])
  x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
  y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]
  
  x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
  y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
  #Splitting data into training(70% data) and testing data(30% data)
  #x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)
  
  x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
  x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)
  TIME_STEPS = 1

  #creating lags for with sentiment scores
  
  x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
  x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)
  
  #creating lags for without sentiment scores
  
  x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
  x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)
  
  x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
  x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))
  
  x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
  x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))
  
  np.random.seed = 1   
  tf.random.set_seed = 2  
  time3=fun_t()
  history=History()
  lstm_model = Sequential()
  lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_without_sentiment_scores.shape[1],x_train_without_sentiment_scores.shape[2])))
  lstm_model.add(keras.layers.Dense(units = 1))
  lstm_model.compile(loss='mean_squared_error', optimizer='adam')
  history = lstm_model.fit(x_train_without_sentiment_scores,y_train_without_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)
  time_lstm = list(time3.times)
  train_lstm = history.history.get('loss')
  val_lstm = history.history.get('val_loss')
  mse=val_lstm[-1]
  print("##############################################################################")
  print('MSE for',tik,'stock excluding Sentiment scores information:',mse)
  predicted_stock_price=lstm_model.predict(x_test_without_sentiment_scores)
  predicted_stock_price = ysc.inverse_transform(predicted_stock_price)

  print("Predicted stock price inverse transformed:",predicted_stock_price[-1])

  #The below data's stock price needs to be compared with prediction for the last row from above print
  print("Previous day data:",tik_data.iloc[-2,:])
  print("Check Adj Close for comparison with the inverse transformed predicted stock price")
  #print("Actual stock price:",tik_data.iloc[-1,:])
  
  
  #Calculating the top mover category using above predictions
  percent_change = ((predicted_stock_price[-1]-tik_data.iloc[-2,4])/tik_data.iloc[-2,4])*100
  print("Percentage change expected next day",percent_change)
  if percent_change > int(defined_topmover_threshold) and percent_change > 0: 
    print(tik,"stock is predicted to be a top mover the next day")
  else: print(tik,"stock might not be a top mover the next day")
  print("##############################################################################")

##############################################################################
MSE for ZION stock excluding Sentiment scores information: 0.5847963690757751
Predicted stock price inverse transformed: [16.983372]
Previous day data: Open         2.412000e+01
High         2.450000e+01
Low          2.386000e+01
Close        2.439000e+01
Adj Close    2.208469e+01
Volume       4.786600e+06
Name: 2016-06-28, dtype: float64
Check Adj Close for comparison with the inverse transformed predicted stock price
Percentage change expected next day [-23.098904]
ZION stock might not be a top mover the next day
##############################################################################
##############################################################################
MSE for KGC stock excluding Sentiment scores information: 0.7469857931137085
Predicted stock price inverse transformed: [13.109369]
Previous day data: Open         5.020000e+00
High         5.110000e+00
Low          4.950000e+00
Close        

# LSTM PREDICTIONS USING SENTIMENT SCORE INFORMATION

In [ ]:
##ZION,KGC,AMD,SOHU,BLIN,WMT,AMGN,NKTR,T,AAPL

for tik in ticker_list:
  tik_data=pd.read_csv(tik+'.csv',index_col=0)
  temp_data=tik_data.drop(['Close'],axis=1)
  
  with_sentiment_data=pd.merge(labels, temp_data, right_index=True, left_index=True)
  #Scaling the variables(x) and variable(y)
  sca = temp_data.iloc[:,:]
  #Standardization of data is required as the dataset consists of variables of different scales
  sc = StandardScaler()
  ysc = StandardScaler()
  scaled_wo_sentiment = sc.fit_transform(sca)
  scaled_wo_sentiment = pd.DataFrame(data=scaled_wo_sentiment,columns=['Open','High','Low','Volume','Adj Close'])
  
  #Y scaling for later inverse transforming predictions
  yscalesenti = with_sentiment_data
  y_with_sentiscal = ysc.fit_transform(yscalesenti.iloc[:,4].to_numpy().reshape(-1,1))
  with_sentiment_data = sc.fit_transform(with_sentiment_data)

  with_sentiment_data = pd.DataFrame(data=with_sentiment_data,columns=['Label','Open','High','Low','Volume','Adj Close'])
  x_without_sentiment_scores = scaled_wo_sentiment.iloc[:,0:4]
  y_without_sentiment_scores = scaled_wo_sentiment.iloc[:,4]
  
  x_with_sentiment_scores = with_sentiment_data.iloc[:,0:5]
  y_with_sentiment_scores = with_sentiment_data.iloc[:,5]
  #Splitting data into training(70% data) and testing data(30% data)
  #x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=1)
  
  x_train_without_sentiment_scores, x_test_without_sentiment_scores, y_train_without_sentiment_scores, y_test_without_sentiment_scores = train_test_split(x_without_sentiment_scores, y_without_sentiment_scores, test_size=0.30,random_state=1)
  x_train_with_sentiment_scores, x_test_with_sentiment_scores, y_train_with_sentiment_scores, y_test_with_sentiment_scores = train_test_split(x_with_sentiment_scores, y_with_sentiment_scores, test_size=0.30,random_state=1)
  TIME_STEPS = 1

  #creating lags for with sentiment scores
  
  x_train_with_sentiment_scores, y_train_with_sentiment_scores = dataset(x_train_with_sentiment_scores,y_train_with_sentiment_scores, y_train_with_sentiment_scores, TIME_STEPS)
  x_test_with_sentiment_scores, y_test_with_sentiment_scores = dataset(x_test_with_sentiment_scores,y_test_with_sentiment_scores, y_test_with_sentiment_scores, TIME_STEPS)
  
  #creating lags for without sentiment scores
  
  x_train_without_sentiment_scores, y_train_without_sentiment_scores = dataset(x_train_without_sentiment_scores,y_train_without_sentiment_scores, y_train_without_sentiment_scores, TIME_STEPS)
  x_test_without_sentiment_scores, y_test_without_sentiment_scores = dataset(x_test_without_sentiment_scores,y_test_without_sentiment_scores, y_test_without_sentiment_scores, TIME_STEPS)
  
  x_train_with_sentiment_scores=x_train_with_sentiment_scores.reshape((x_train_with_sentiment_scores.shape[0], 1,x_train_with_sentiment_scores.shape[1]))
  x_test_with_sentiment_scores=x_test_with_sentiment_scores.reshape((x_test_with_sentiment_scores.shape[0], 1,x_test_with_sentiment_scores.shape[1]))
  
  x_train_without_sentiment_scores=x_train_without_sentiment_scores.reshape((x_train_without_sentiment_scores.shape[0], 1,x_train_without_sentiment_scores.shape[1]))
  x_test_without_sentiment_scores=x_test_without_sentiment_scores.reshape((x_test_without_sentiment_scores.shape[0], 1,x_test_without_sentiment_scores.shape[1]))
  
  np.random.seed = 1   
  tf.random.set_seed = 2  
  time3=fun_t()
  history=History()
  lstm_model = Sequential()
  lstm_model.add(keras.layers.LSTM(units = 1, input_shape = (x_train_with_sentiment_scores.shape[1],x_train_with_sentiment_scores.shape[2])))
  lstm_model.add(keras.layers.Dense(units = 1))
  lstm_model.compile(loss='mean_squared_error', optimizer='adam')
  history = lstm_model.fit(x_train_with_sentiment_scores,y_train_with_sentiment_scores, epochs= 150, shuffle = False, batch_size = 7, callbacks=[history,time3],validation_split = 0.3,verbose = 0)
  time_lstm = list(time3.times)
  train_lstm = history.history.get('loss')
  val_lstm = history.history.get('val_loss')
  mse=val_lstm[-1]
  print("##############################################################################")
  print('MSE for',tik,'stock using Sentiment scores:',mse)
  predicted_stock_price=lstm_model.predict(x_test_with_sentiment_scores)
  predicted_stock_price = ysc.inverse_transform(predicted_stock_price)

  print("Predicted stock price inverse transformed:",predicted_stock_price[-1])

  #The below data's stock price needs to be compared with prediction for the last row from above print
  print("Previous day data:",tik_data.iloc[-2,:])
  #print("Actual stock price:",tik_data.iloc[-1,:])
  print("Check Adj Close for comparison with the inverse transformed predicted stock price")
  #Calculating the top mover category using above predictions
  percent_change = ((predicted_stock_price[-1]-tik_data.iloc[-2,4])/tik_data.iloc[-2,4])*100
  print("Percentage change expected next day",percent_change)
  if percent_change > int(defined_topmover_threshold) and percent_change > 0: 
    print(tik,"stock is predicted to be a top mover the next day")
  else: print(tik,"stock might not be a top mover the next day")
  print("##############################################################################")

##############################################################################
MSE for ZION stock using Sentiment scores: 0.7058767080307007
Predicted stock price inverse transformed: [17.7697]
Previous day data: Open         2.412000e+01
High         2.450000e+01
Low          2.386000e+01
Close        2.439000e+01
Adj Close    2.208469e+01
Volume       4.786600e+06
Name: 2016-06-28, dtype: float64
Check Adj Close for comparison with the inverse transformed predicted stock price
Percentage change expected next day [-19.538395]
ZION stock might not be a top mover the next day
##############################################################################
##############################################################################
MSE for KGC stock using Sentiment scores: 0.745173990726471
Predicted stock price inverse transformed: [12.9155245]
Previous day data: Open         5.020000e+00
High         5.110000e+00
Low          4.950000e+00
Close        4.960000e+00
Adj Close    4.922116